# Bad Photo Generator

### Step 1: Generating bad photos from good photos

![image info](bad_images.png)

#### Choosing random images from a folder

In [1]:
# loading libraries
import cv2
import numpy as np
import os, random
from PIL import Image

In [2]:
def choose_photos(folder, n_photos):
    number_files = len(os.listdir(folder))
    if number_files < n_photos:
        return 'number of photos is larger than the photos in the folder'
    chosen_photos = set()
    while len(chosen_photos) < n_photos:
        chosen_photos.add(random.choice(os.listdir(folder)))
    # chosen_photos = {random.choice(os.listdir(folder)) for i in range(n_photos)}
    return chosen_photos

In [3]:
folder = r"flickr30k_images"
chosen_photos = choose_photos(folder, 1000)

In [4]:
len(chosen_photos)

1000

#### Creating "good" and "bad" folders

In [5]:
try:
    os.mkdir('good') 
except OSError as error: 
    print(error)

In [6]:
try:
    os.mkdir('bad')
except OSError as error: 
    print(error)

#### Copying the random chosen images into the "good" folder

In [7]:
import shutil

In [8]:
for photo in chosen_photos:
    shutil.copy(folder + '\\' + photo, 'good')

#### Generating bad photos

In [9]:
# motion filter
#size - in pixels, size of motion blur
#angle - in degrees, direction of motion blur
def apply_motion_blur(image, size, angle):
    k = np.zeros((size, size), dtype=np.float32)
    k[ (size-1)// 2 , :] = np.ones(size, dtype=np.float32)
    k = cv2.warpAffine(k, cv2.getRotationMatrix2D( (size / 2 -0.5 , size / 2 -0.5 ) , angle, 1.0), (size, size) )
    k = k * ( 1.0 / np.sum(k) )
    return cv2.filter2D(image, -1, k)

In [10]:
# illumination filter
def adjust_gamma(image, gamma=0.5):
    invGamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** invGamma) * 255 for i in np.arange(0, 256)])
    return cv2.LUT(image.astype(np.uint8), table.astype(np.uint8))

In [11]:
# light source filter
def fake_light(image, size=0.5):
    img_size = (image.shape[0], image.shape[1], 3)
    mask = np.zeros(img_size, np.uint8)

    lightsource = cv2.imread('lightsource.png')
    lightsource_size = (int(img_size[0]*size), int(img_size[1]*size))
    lightsource = cv2.resize(lightsource, lightsource_size)
    
    mask = Image.fromarray(np.uint8(mask))
    lightsource = Image.fromarray(np.uint8(lightsource))

    x, y = (random.randint(0, img_size[0]-lightsource_size[0]), random.randint(0, img_size[1]-lightsource_size[1]))

    mask.paste(lightsource, (x, y))
    mask = np.asarray(mask)

    return cv2.addWeighted(img, 1, mask, 1, 0)

In [12]:
def apply_filter(image, filter_name='blur'):
    if filter_name == 'blur':
        return apply_motion_blur(image, 10, 90)
    if filter_name == 'bright':
        return adjust_gamma(image, 3)
    if filter_name == 'dark':
        return adjust_gamma(image, 0.3)
    if filter_name == 'light':
        return fake_light(image)

In [13]:
for img_file in chosen_photos:
    img = cv2.imread('good\\' + img_file)
    cv2.imwrite('bad\\' + img_file[:-4] + '_blur.jpg', apply_filter(img, filter_name='blur'))
    cv2.imwrite('bad\\' + img_file[:-4] + '_bright.jpg', apply_filter(img, filter_name='bright'))
    cv2.imwrite('bad\\' + img_file[:-4] + '_dark.jpg', apply_filter(img, filter_name='dark'))
    cv2.imwrite('bad\\' + img_file[:-4] + '_light.jpg', apply_filter(img, filter_name='light'))